##### 0.5: Change colab runtime type to "GPU"

# NetZooPy Tutorial: running Panda and Lioness using GPU and precision flags

#1. Setup Environment: download, install & import latest packages

In [0]:
import os
import s3fs
import pandas as pd
import numpy as np
from psutil import *

In [0]:
!nvcc --version

###  
##1.1 Configure cupy for installed Cuda version
1.   use cuda version based on above release (*e.g.* release 10.1= cuda101)
2.   install cupy version 7.4.0 or above 



In [2]:
!pip install cupy-cuda101==7.4.0
import cupy as cp
# print(cp.__version__)
!pip freeze | grep cupy

     |████████████████████████████████| 361.9MB 43kB/s 
  Found existing installation: cupy-cuda101 6.5.0
    Uninstalling cupy-cuda101-6.5.0:
      Successfully uninstalled cupy-cuda101-6.5.0
cupy-cuda101==7.4.0


In [0]:
!rm -rf netZooPy

In [0]:
!git clone --single-branch --branch devel https://github.com/netZoo/netZooPy.git
os.chdir('netZooPy')
!pip install -e .
os.chdir('..')

In [0]:
import netZooPy
from netZooPy.panda import Panda
from netZooPy.lioness import Lioness

#2. Load test data we fetch the data from the [GRAND database](https://grand.networkmedicine.org/download/) 

In [0]:
LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/cells/ppi/LCL_ppi.txt',sep='\t',header=None)
LCL_expression=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/expression/Hugo_exp1_lcl.txt',sep='\t',header=None)
LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/gpuPANDA/Hugo_motifCellLine_reduced.txt',sep='\t',header=None)
LCL_ppi.to_csv('LCL_ppi.txt',sep='\t',index=False)
LCL_expression.to_csv('Hugo_exp1_lcl.txt',sep='\t',index=False)
LCL_motif.to_csv('Hugo_motifCellLine_reduced.txt',sep='\t',index=False)

In [0]:
# os.getcwd()
expression_data='Hugo_exp1_lcl.txt'
motif_data='Hugo_motifCellLine_reduced.txt'
ppi_data='LCL_ppi.txt'


#3. Run Panda with GPU & precision flags

In [0]:
panda_obj = Panda(expression_data, motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=True, keep_expression_matrix = True,modeProcess = 'intersection')

In [0]:
panda_obj.save_panda_results('single_cpu_panda.txt')

In [0]:
panda_obj = Lioness(panda_obj,computing='gpu',start=1, end=5, save_dir='lioness_output', save_fmt='txt')#,save_tmp=False,save_memory = True, remove_missing=True, keep_expression_matrix = False,modeProcess = 'intersection')

##GPU and CPU info:

In [0]:
!nvidia-smi

In [0]:
!cat /proc/cpuinfo

In [17]:
panda_obj.panda_network.shape

(1604, 14443)